<a href="https://colab.research.google.com/github/anuragsaraf1912/Babbar450/blob/master/getMarketData.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Market Data Analysis

## Importing libraries

In [1]:
import pandas as pd
import numpy as np
import yfinance as yf
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from IPython.display import clear_output

#Getting the widgets objects
import ipywidgets as widgets
from ipywidgets import HBox, VBox

## Functions

#### Graph Generating Functions

In [157]:
def plotCandleStick(data):
  """Plots candlestick pattern using the Plotly library"""

  candlestick = go.Candlestick(
    x=data["Date"],
    open=data["Open"],
    high=data["High"],
    low=data["Low"],
    close=data["Close"],
    name="Price",
    increasing_line_color='green',
    decreasing_line_color='red',
    hovertext=[f"Returns: {round(row['Daily Returns'],2)}% <br> Volume: {round(row['Volume']/1000,2)}K <br> Open:{round(row['Open'],2)} <br> High:{round(row['High'],2)} <br> Low:{round(row['Low'],2)} <br> Close:{round(row['Close'],2)}" for index, row in data.iterrows()],
    hoverinfo="text"  # Display hovertext
  )

  # Create volume trace
  volume = go.Bar(
      x=data["Date"],
      y=data["Volume"],
      name="Volume",
      marker=dict(color='rgba(0, 0, 255, 0.3)')
  )

  # Create figure

  fig = make_subplots(specs=[[{"secondary_y": True}]])
  fig.add_trace(
      candlestick,
      secondary_y=False,
  )

  fig.add_trace(
      volume,
      secondary_y=True,
  )

  # Update layout
  fig.update_layout(
      title="Candlestick Chart with Volume",
      height = 1000,
      width = 2000,
      xaxis=dict(
          title="Date",
      ),
  )

  # Configure the y-axis for volume
  fig.update_yaxes(title_text="Price", secondary_y=False, )
  fig.update_yaxes(title_text="Volume", secondary_y=True, range =[0, 5*max(data["Volume"])])

  # Show figure
  fig.show()

###################################################################################################################################################################

def makeHistogram(df):
  """Creates a histogram from the data. Requires a 'Daily Returns' column"""
  fig = go.Figure(data=[go.Histogram(x=df['Daily Returns'])])

  # Update layout for better appearance
  fig.update_layout(
      title="Daily Returns Distribution",
      xaxis_title="Returns",
      yaxis_title="Count",
      bargap=0.1,  # Gap between bars
      bargroupgap=0.1,  # Gap between groups of bars
      height = 700,
      width = 1000
  )

  # Show the figure (interactive)
  fig.show()

###################################################################################################################################################################

def makeLinePlot(sortedDF):
  """Create a quartle graph from the returns data. Requires a percentile and Daily returns column"""

  # Create a line graph using go.Scatter
  lineGraph = go.Figure()

  x_values, y_values = round(sortedDF['Daily Returns'],3),  round(sortedDF['percentile'],3)

  # Add the line graph trace
  lineGraph.add_trace(go.Scatter(x=x_values, y=y_values, mode='lines', name='Percentile Line'))


  # Customize the layout
  lineGraph.update_layout(
      title="Percentile line for the Stock Daily Returns",
      xaxis_title="Daily Returns",
      yaxis_title="Percentile",
      height=700,
      width=1000,
  )
  # Show the figure
  lineGraph.show()

###################################################################################################################################################################

def makeTimeSeries(df):
  """Create a time series graph from the data. Requires a 'Daily Returns' and 'Date' columns"""
  fig = go.Figure()

  # Add the returns line
  fig.add_trace(go.Scatter(
      x=df['Date'],
      y=round(df['Daily Returns'],3),
      mode='lines',
      name='Daily Returns'#,
      # hovertext=[f"Date: {date.date()}<br>Return: {return_val:.2%}" for date, return_val in zip(df['Date'], df['Returns'])],
      # hoverinfo='text'
  ))

  # Customize the layout
  fig.update_layout(
      title="Time Series of Daily Returns",
      xaxis_title="Date",
      yaxis_title="Returns",
      xaxis_rangeslider_visible=False,
      height = 700,
      width = 1000
  )

  # Add a horizontal dashed line at y=0
  fig.add_shape(
      type='line',
      x0=df['Date'].iloc[0],  # Starting x value (beginning of the x-axis)
      y0=0,                   # y value (height of the line)
      x1=df['Date'].iloc[-1], # Ending x value (end of the x-axis)
      y1=0,                   # y value (height of the line)
      line=dict(color='red', width=1, dash='dash')  # Line style
  )

  # Show the figure
  fig.show()


#### Data Logic

In [140]:
def getTicker(tick):
  """Logic to modify ticker before sending it to the Yahoo finance API"""
  if tick == '^NSEI':
    return '^NSEI'
  else:
    return tick + ".NS"

def getData(ticker, startDate, endDate):
  """Logic to get data from the Yahoo finance API"""
  df = (yf.download(ticker, startDate, endDate)).reset_index()
  return df

def getDailyReturns(df):
  """Logic to get daily returns from the data"""
  df['Daily Returns'] = 100*np.log(df['Adj Close']/df['Adj Close'].shift(1))
  df.dropna(inplace=True)
  return df

def genSortedPercentile(df):
  """Logic to generate the percentile column"""
  sortedDF = df.sort_values('Daily Returns')
  sortedDF['percentile'] =  1
  sortedDF['percentile'] = sortedDF['percentile'].cumsum()/len(sortedDF)
  return sortedDF


#### Event functions

In [ ]:
def on_value_change(change):
    if change['type'] == 'change' and change['name'] == 'value':
        startDate = start_date_picker.value
        endDate = end_date_picker.value
        ticker = getTicker(dropdown.value)
        if startDate and endDate and ticker:
          global df, sortedDF
          df = getData(ticker, startDate, endDate)
          df = getDailyReturns(df)
          sortedDF = genSortedPercentile(df)
          clear_output(wait = True)
          display(widgets.HBox([dropdown, start_date_picker, end_date_picker])
)

## Widgets

#### Initiate Widgets

In [163]:
# Date Picker Widgets
start_date_picker = widgets.DatePicker(description='Start Date:')
end_date_picker = widgets.DatePicker(description='End Date:')

# Text Entry Widget
options = ['^NSEI', 'HDFCBANK', 'TCS']

# Create the dropdown widget
dropdown = widgets.Dropdown(
    options=options,
    description='Select stock:'
)

# Observing changes
start_date_picker.observe(on_value_change)
end_date_picker.observe(on_value_change)
dropdown.observe(on_value_change)

#### Stock and Date Selector

In [164]:
# Displaying widgets
widgets.HBox([dropdown, start_date_picker, end_date_picker])
#Some stocks to look into: HDFCBANK, WSTCSTPAPR

# Analysis

In [165]:
plotCandleStick(df)

In [166]:
makeLinePlot(sortedDF)

In [167]:
makeHistogram(df)

In [168]:
makeTimeSeries(df)

In [169]:
sortedDF

,Date,Open,High,Low,Close,Adj Close,Volume,Daily Returns,percentile
102,2024-06-04,23179.500000,23179.500000,21281.449219,21884.500000,21884.500000,1006100,-6.112420,0.005464
144,2024-08-05,24302.849609,24350.050781,23893.699219,24055.599609,24055.599609,487000,-2.715175,0.010929
12,2024-01-17,21647.250000,21851.500000,21550.449219,21571.949219,21571.949219,456000,-2.111577,0.016393
15,2024-01-23,21716.699219,21750.250000,21192.599609,21238.800781,21238.800781,449700,-1.790010,0.021858
85,2024-05-09,22224.800781,22307.750000,21932.400391,21957.500000,21957.500000,331300,-1.559001,0.027322
...,...,...,...,...,...,...,...,...,...
18,2024-01-29,21433.099609,21763.250000,21429.599609,21737.599609,21737.599609,376700,1.786997,0.978142
171,2024-09-12,25059.650391,25433.349609,24941.449219,25388.900391,25388.900391,380100,1.870362,0.983607
105,2024-06-07,22821.849609,23320.199219,22789.050781,23290.150391,23290.150391,473600,2.033183,0.989071
101,2024-06-03,23337.900391,23338.699219,23062.300781,23263.900391,23263.900391,569400,3.202403,0.994536
